In [6]:
import json
import requests
import os
os.getcwd()
os.chdir('/Users/justinniestroy-admin/Documents/Round2/Fairscape_Repos/mds_python/src')

root_url = "http://localhost:8080/"


In [7]:
from fairscape_mds.mds.config import *
mongo_config = get_mongo_config()
mongo_client = get_mongo_client()
mongo_db = mongo_client[mongo_config.db]
rocrate_collection = mongo_db[mongo_config.rocrate_collection]
identifier_collection = mongo_db[mongo_config.identifier_collection]

# ROCrate

## Bugs
1. GetMetadata can look in the wrong spot for json file if folder and zip file have different names
2. Same error as above happens if you have a . in the zip folder name

In [8]:
from fairscape_mds.mds.models.rocrate import *
ListROCrates(rocrate_collection)

{'rocrates': [{'@id': 'ark:59852/rocrate-name-for-pipeline-run-PucM24Z',
   'name': 'Name for pipeline run',
   'description': 'Name of lab or group. Ex: Ideker Name of funding source or project Name of release. Example: 0.1 alpha Name of cell line. Ex: U2OS Name of treatment, Ex: untreated Name of gene set. Example chromatin Name for pipeline run AP-MS Embedding AP-MS embedding',
   'keywords': ['Name of lab or group. Ex: Ideker',
    'Name of funding source or project',
    'Name of release. Example: 0.1 alpha',
    'Name of cell line. Ex: U2OS',
    'Name of treatment, Ex: untreated',
    'Name of gene set. Example chromatin',
    'Name for pipeline run',
    'AP-MS Embedding',
    'AP-MS',
    'embedding'],
   'sourceOrganization': None}]}

In [12]:
rocrate_id = 'ark:59852/rocrate-name-for-pipeline-run-PucM24Z'

crate = ROCrate.construct(guid=rocrate_id)
read_status = crate.read(rocrate_collection)

In [13]:
crate

ROCrate(guid='ark:59852/rocrate-name-for-pipeline-run-PucM24Z', metadataType='https://schema.org/Dataset', context={'@vocab': 'https://schema.org/', 'evi': 'https://w3id.org/EVI#'}, url=None, additionalType='ROCrate', sourceOrganization=None, contentURL=None, distribution={'extractedROCrateBucket': ['default', 'default', 'default', 'default', 'default', 'default', 'default', 'default', 'default', 'default', 'default', 'default'], 'archivedROCrateBucket': 'rocrate', 'extractedObjectPath': ['02ca2397-78b3-4a77-b717-efc83b73f6c2/2_ppi_embedding', '02ca2397-78b3-4a77-b717-efc83b73f6c2/2_ppi_embedding/.DS_Store', '02ca2397-78b3-4a77-b717-efc83b73f6c2/__MACOSX/2_ppi_embedding/._.DS_Store', '02ca2397-78b3-4a77-b717-efc83b73f6c2/2_ppi_embedding/test', '02ca2397-78b3-4a77-b717-efc83b73f6c2/2_ppi_embedding/error.log', '02ca2397-78b3-4a77-b717-efc83b73f6c2/2_ppi_embedding/output.log', '02ca2397-78b3-4a77-b717-efc83b73f6c2/2_ppi_embedding/task_1710542634_finish.json', '02ca2397-78b3-4a77-b717-efc8

In [14]:
from fastapi import (
    APIRouter, 
    UploadFile, 
#    Form, 
    File, 
#    Response, 
#    Header
)

In [14]:
from jinja2 import Environment, FileSystemLoader
from pydantic import BaseModel, Field
from typing import List, Optional

from fairscape_mds.mds.models.user import User
from fairscape_mds.mds.models.fairscape_base import *
import json

# Create an instance of the User class with some data
user = User(
    guid = "ark:99999/fake-id",
    name = 'fake user',
    email="user@example.com",
    password="password123",
    organizations=["Org1", "Org2"],
    projects=["Project1", "Project2"],
    datasets=["Dataset1", "Dataset2"],
    rocrates=["ROCrate1", "ROCrate2"],
    software=["Software1", "Software2"],
    computations=["Computation1", "Computation2"]
)

# Set up the Jinja2 environment and load the template
env = Environment(loader=FileSystemLoader('../'))

import re

def add_link(value):
    """For values that match ark or look like urls add a hyperlink"""
    url_pattern = r'^(http|https)://[^\s]+'
    if re.match(IdentifierPattern, value):
        return f'<a href="https://fairscape.pods.uvarc.io/{value}">{value}</a>'
    elif re.match(url_pattern, value):
        return f'<a href="{value}">{value}</a>'
    return value

env.filters['add_link'] = add_link
template = env.get_template('./development notebooks/vizualization/user_template.html')

# Render the template with the user data
rendered_html = template.render(user=user, json = json, type = 'User')

# Print or save the rendered HTML
with open('example.html', 'w') as f:
    f.write(rendered_html)


In [12]:
from fairscape_mds.mds.models.software import Software
sample_software = Software(
    guid = 'ark:99999/fake-software',
    name = 'Software Example',
    description = 'Fake Software to show template working.',
    owner="John Doe",
    distribution=["https://example.com/software/download", "https://example.com/software/mirror"],
    usedBy=['ark:99999/fake-software','ark:99999/fake-computation'],
    sourceOrganization='ark:99999/fake-org',
    includedInDataCatalog="ark:99999/fake-catalog"
)

template = env.get_template('./development notebooks/vizualization/software_template.html')

# Render the template with the user data
rendered_html = template.render(software=sample_software, json = json, type = 'Software')

# Print or save the rendered HTML
with open('example.html', 'w') as f:
    f.write(rendered_html)


In [1]:
import json
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

root_url = "http://localhost:8080/"
crate_path = '/Users/justinniestroy-admin/Documents/Round2/PreMO/premo_rocrate.zip'

mp_encoder = MultipartEncoder(
    fields={        
        'file': ('premo_rocrate.zip', open(crate_path, 'rb'), 'application/zip')
    }
)

# upload a rocrate to minio object store
rocrate_transfer = requests.post(
    url=root_url + 'rocrate/upload',
    data=mp_encoder,                              
    # The MultipartEncoder provides the content-type header with the boundary:
    headers={'Content-Type': mp_encoder.content_type}
)

rocrate_transfer.json()

{'created': {'@id': 'ark:59852/rocrate-premo-rocrate-eVHFS456',
  '@type': 'Dataset',
  'name': 'PreMo ROCrate'}}

In [1]:
schema = {
    "@context": {
        "@vocab": "https://schema.org/",
        "EVI": "https://w3,org/EVI#"
    },
    "@id": "ark:59852/schema-predictive-modeling-izNjXSs",
    "@type": "EVI:Schema",
    "name": "PreMo RedCAP Schema",
    "description": "Schema for predicitive modeling RedCAP DB.",
    "properties": {
        "subject_number": {
            "description": "Subject Number",
            "index": 0,
            "type": "number",
            "minimum": 0.0,
            "maximum": 99999.0
        },
        "demograph_site_id": {
            "description": "The variable demograph_site_id is a unique identifier used to associate demographic data with a specific collection site.",
            "index": 1,
            "type": "string"
        },
        "demograph_site_num": {
            "description": "Site Number. UVA = 1, WU = 2, CU = 3, UAB = 4",
            "index": 2,
            "type": "string"
        },
        "vsdata_yn": {
            "description": "Does this patient have a vital sign data file available for analysis? ",
            "index": 3,
            "type": "boolean"
        },
        "demograph_birth_date": {
            "description": "Birth Date",
            "index": 4,
            "type": "string",
            "value-url": "https://schema.org/birthDate"
        },
        "demograph_birthtime": {
            "description": "Time of birth",
            "index": 5,
            "type": "string"
        },
        "demograph_birth_ga_weeks": {
            "description": "Gestational Age in Weeks at Birth",
            "index": 6,
            "type": "number",
            "minimum": 22.0,
            "maximum": 39.0
        },
        "demograph_birth_ga_days": {
            "description": "Gestational Age days at Birth",
            "index": 7,
            "type": "number",
            "minimum": 0.0,
            "maximum": 6.0
        },
        "demograph_gender": {
            "description": "Gender",
            "index": 8,
            "type": "string",
            "pattern": "^(Female|Male)$"
        },
        "demograph_race": {
            "description": "Race",
            "index": 9,
            "type": "string",
            "pattern": "^(White\\ or\\ Caucasian|Black\\ or\\ African\\ American|Asian|American\\ Indian\\ or\\ Alaska\\ Native|Native\\ Hawaiian\\ or\\ Other\\ Pacific\\ Islander|Unknown/Not\\ Reported)$"
        },
        "demograph_ethnicity": {
            "description": "Ethnicity:",
            "index": 10,
            "type": "string",
            "pattern": "^(Hispanic\\ or\\ Latino|Non\\-Hispanic\\ or\\ Non\\-Latino|Unknown/Not\\ Reported)$"
        },
        "dem_adi": {
            "description": "National ADI Percentile of the birth address",
            "index": 11,
            "type": "integer",
            "minimum": 0,
            "maximum": 101
        },
        "multiples": {
            "description": "Multiple gestation: (twin, triplet, etc.) ",
            "index": 12,
            "type": "boolean"
        },
        "multiple_id": {
            "description": "Subject ID of first-born infant in multiple gestation:",
            "index": 13,
            "type": "number"
        },
        "delivery_birth_weight": {
            "description": "Birth Weight (grams)",
            "index": 14,
            "type": "number",
            "minimum": 200.0,
            "maximum": 5000.0
        },
        "delivery_apgar_1_min": {
            "description": "Apgar Score 1 Minute",
            "index": 15,
            "type": "number",
            "minimum": 0.0,
            "maximum": 9.0,
            "value-url": "http://purl.bioontology.org/ontology/RCD/X76WK"
        },
        "delivery_apgar_5_min": {
            "description": "Apgar Score 5 Minutes",
            "index": 16,
            "type": "number",
            "value-url": "http://purl.bioontology.org/ontology/RCD/X76WK"
        },
        "delivery_apgar_10_min": {
            "description": "Apgar Score 10 Minutes",
            "index": 17,
            "type": "number",
            "value-url": "http://purl.bioontology.org/ontology/RCD/X76WK"
        },
        "delivery_steroid_dose": {
            "description": "Did mother receive at least one dose of steroids prior to delivery? ",
            "index": 18,
            "type": "boolean"
        },
        "delivery_mode": {
            "description": "Mode of Delivery",
            "index": 19,
            "type": "string",
            "pattern": "^(C\\-section|Vaginal\\ Delivery|Unknown)$"
        },
        "delivery_admin_type": {
            "description": "Admission Type",
            "index": 20,
            "type": "string",
            "pattern": "^(Inborn|Outborn|Unknown)$"
        },
        "demograph_admit_date": {
            "description": "Admission Date",
            "index": 21,
            "type": "string"
        },
        "delivery_admit_age": {
            "description": "Age at Admission (days)",
            "index": 22,
            "type": "string"
        },
        "eos_yn": {
            "description": "Was a blood culture sent within 3 days of birth? ",
            "index": 23,
            "type": "string",
            "pattern": "^(Yes|No|Unknown)$"
        },
        "eos_positive": {
            "description": "If yes, was the culture positive? ",
            "index": 24,
            "type": "boolean",
            "value-type": "http://purl.bioontology.org/ontology/RCD/X77bV"
        },
        "eos_organism": {
            "description": "If positive what organism was present?",
            "index": 25,
            "type": "string",
            "pattern": "^(S\\.\\ Epi\\ or\\ CONS|E\\.\\ coli|S\\.\\ Aureus|Group\\ B\\ Streptococcus\\ sp\\.|Klebsiella\\ sp\\.|Pseudomonas\\ sp\\.|Enterococcus\\ sp\\.|Enterobacter\\ sp\\.|Other)$",
            "value-url": "http://purl.bioontology.org/ontology/RCD/X79pY"
        },
        "eos_organism_other": {
            "description": "Organism if 'Other': ",
            "index": 26,
            "type": "string",
            "value-url": "http://purl.bioontology.org/ontology/RCD/X79pY"
        },
        "eos_abx_days": {
            "description": "Number of consecutive days treated with antibiotics following a blood culture obtained for suspected EOS",
            "index": 27,
            "type": "integer"
        },
        "eos_contaminant": {
            "description": "Was the positive culture result considered a contaminant? ",
            "index": 28,
            "type": "string",
            "pattern": "^(Yes|No|Unknown)$"
        },
        "los_num": {
            "description": "Number of blood cultures sent AFTER 3 days",
            "index": 29,
            "type": "integer"
        },
        "ivh_grade": {
            "description": "Highest grade IVH on any HUS, any side:",
            "index": 30,
            "type": "string",
            "pattern": "^(0|1|2|3|4)$"
        },
        "bpd_36_weeks": {
            "description": "Did the infant require supplemental oxygen at 36 weeks PMA? ",
            "index": 31,
            "type": "string",
            "pattern": "^(Yes|No|Unknown)$"
        },
        "rop_laser_avastin": {
            "description": "Was the infant treated with laser and/or Avastin? ",
            "index": 32,
            "type": "string",
            "pattern": "^(Yes|No|Unknown)$"
        },
        "bayley_yn": {
            "description": "Was a Bayley exam performed at 18-24 months old? ",
            "index": 33,
            "type": "string",
            "pattern": "^(No|Yes|Unknown)$"
        },
        "bayley_date": {
            "description": "Date of Bayley Exam",
            "index": 34,
            "type": "string"
        },
        "bayley_version": {
            "description": "Bayley exam version",
            "index": 35,
            "type": "string",
            "pattern": "^(Bayley\\ III|Bayley\\ IV|Unknown)$"
        },
        "bayley_cog": {
            "description": "Cognitive score",
            "index": 36,
            "type": "integer"
        },
        "bayeley_lang": {
            "description": "Language score",
            "index": 37,
            "type": "integer"
        },
        "bayley_motor": {
            "description": "Motor Score",
            "index": 38,
            "type": "integer"
        },
        "culture_date": {
            "description": "Date blood culture drawn",
            "index": 39,
            "type": "string"
        },
        "culture_time": {
            "description": "Time blood culture drawn",
            "index": 40,
            "type": "string"
        },
        "culture_age": {
            "description": "Age at culture:",
            "index": 41,
            "type": "string"
        },
        "sep_ev1": {
            "description": "Was the blood culture positive? ",
            "index": 42,
            "type": "boolean"
        },
        "culture_result": {
            "description": "Blood culture result",
            "index": 43,
            "type": "string"
        },
        "sep_ev1_blood_org_other": {
            "description": "Organism if Other: ",
            "index": 44,
            "type": "string"
        },
        "abx_days": {
            "description": "Was this event treated with IV antibiotics for > or = 5 days? ",
            "index": 45,
            "type": "boolean"
        },
        "abx_num": {
            "description": "Number of consecutive antibiotic treatment days related to this blood culture event: ",
            "index": 46,
            "type": "integer"
        },
        "culture_vent_yn": {
            "description": "Was the infant on a ventilator at the time of this blood culture?",
            "index": 47,
            "type": "boolean"
        },
        "culture_pos_diag": {
            "description": "Blood culture positive event diagnosis:",
            "index": 48,
            "type": "string",
            "pattern": "^(Septicemia|Contaminant|Urosepsis|NEC\\ with\\ bacteremia|SIP\\ with\\ bacteremia|Meningitis\\ with\\ bacteremia|Pneumonia\\ with\\ bacteremia|Unknown|Other|Repeat\\ culture\\ on\\ antibiotics)$"
        },
        "culture_pos_diag_other": {
            "description": "'Other' culture positive event diagnosis",
            "index": 49,
            "type": "string"
        },
        "culture_neg_diag": {
            "description": "Blood culture negative event diagnosis: ",
            "index": 50,
            "type": "string",
            "pattern": "^(Clinical\\ sepsis|Sepsis\\ ruled\\ out|UTI|NEC|SIP|Pneumonia|Meningitis|Skin|Other|Unknown|Repeat\\ culture\\ on\\ antibiotics)$"
        },
        "culture_neg_diag_other": {
            "description": "'Other' culture-negative event diagnosis: ",
            "index": 51,
            "type": "string"
        },
        "nec_surgery": {
            "description": "Was a surgical intervention performed? ",
            "index": 52,
            "type": "boolean"
        },
        "nec_surgery_type": {
            "description": "If \"Yes\", Type of Procedure",
            "index": 53,
            "type": "string"
        },
        "nec_surg_date": {
            "description": "Date of Procedure: ",
            "index": 54,
            "type": "string"
        },
        "sip_surgery": {
            "description": "Was an surgical intervention performed? ",
            "index": 55,
            "type": "boolean"
        },
        "sip_surgery_type": {
            "description": "If Yes what type of procedure was used?",
            "index": 56,
            "type": "string"
        },
        "sip_surg_date": {
            "description": "Date of procedure",
            "index": 57,
            "type": "string"
        },
        "other_culture_types": {
            "description": "Other culture(s) sent related to this blood culture evaluation: ",
            "index": 58,
            "type": "string"
        },
        "urine_result": {
            "description": "Urine culture result: ",
            "index": 59,
            "type": "string",
            "pattern": "^(Negative\\ or\\ <\\ 10,000\\ CFU|S\\.\\ Epi\\ or\\ CONS|E\\.\\ coli|S\\.\\ Aureus|Group\\ B\\ Streptococcus\\ sp\\.|Klebsiella\\ sp\\.|Pseudomonas\\ sp\\.|Enterococcus\\ sp\\.|Enterobacter\\ sp\\.|Other)$"
        },
        "urine_other": {
            "description": "Organism if urine culture result is 'other': ",
            "index": 60,
            "type": "string"
        },
        "csf_result": {
            "description": "CSF culture result: ",
            "index": 61,
            "type": "string",
            "pattern": "^(Negative|S\\.\\ Epi\\ or\\ CONS|E\\.\\ coli|S\\.\\ Aureus|Group\\ B\\ Streptococcus\\ sp\\.|Klebsiella\\ sp\\.|Pseudomonas\\ sp\\.|Enterococcus\\ sp\\.|Enterobacter\\ sp\\.|Other)$"
        },
        "csf_other": {
            "description": "Organism if CSF result is 'other'",
            "index": 62,
            "type": "string"
        },
        "sputum_result": {
            "description": "Sputum or tracheal aspirate result: ",
            "index": 63,
            "type": "string",
            "pattern": "^(Negative|S\\.\\ Epi\\ or\\ CONS|E\\.\\ coli|S\\.\\ Aureus|Group\\ B\\ Streptococcus\\ sp\\.|Klebsiella\\ sp\\.|Pseudomonas\\ sp\\.|Enterococcus\\ sp\\.|Enterobacter\\ sp\\.|Other)$"
        },
        "sputum_other": {
            "description": "Organism if sputum culture result is 'other': ",
            "index": 64,
            "type": "string"
        },
        "skin_result": {
            "description": "Skin or wound culture result: ",
            "index": 65,
            "type": "string",
            "pattern": "^(Negative|S\\.\\ Epi\\ or\\ CONS|E\\.\\ coli|S\\.\\ Aureus|Group\\ B\\ Streptococcus\\ sp\\.|Klebsiella\\ sp\\.|Pseudomonas\\ sp\\.|Enterococcus\\ sp\\.|Enterobacter\\ sp\\.|Other)$"
        },
        "skin_other": {
            "description": "Organism if skin culture result is 'other': ",
            "index": 66,
            "type": "string"
        },
        "peritoneal_result": {
            "description": "Peritoneal fluid culture result: ",
            "index": 67,
            "type": "string",
            "pattern": "^(Negative|S\\.\\ Epi\\ or\\ CONS|E\\.\\ coli|S\\.\\ Aureus|Group\\ B\\ Streptococcus\\ sp\\.|Klebsiella\\ sp\\.|Pseudomonas\\ sp\\.|Enterococcus\\ sp\\.|Enterobacter\\ sp\\.|Other)$"
        },
        "other_type": {
            "description": "'Other' culture type sent: ",
            "index": 68,
            "type": "string"
        },
        "other_result": {
            "description": "Other type culture result: ",
            "index": 69,
            "type": "string"
        },
        "viral_type": {
            "description": "What viral studies were sent? ",
            "index": 70,
            "type": "string"
        },
        "viral_result1": {
            "description": "Respiratory virus PCR result: ",
            "index": 71,
            "type": "string",
            "pattern": "^(Positive|Negative|Unknown)$",
            "value-url": "http://purl.allotrope.org/ontologies/result#AFR_0002279"
        },
        "viral_result_2": {
            "description": "GI pathogen result: ",
            "index": 72,
            "type": "string",
            "pattern": "^(Positive|Negative|Unknown)$"
        },
        "viral_result_3": {
            "description": "HSV PCR from CSF result: ",
            "index": 73,
            "type": "string",
            "pattern": "^(Positive|Negative|Unknown)$",
            "value-url": "http://purl.bioontology.org/ontology/OMIM/MTHU060433"
        },
        "viral_result_4": {
            "description": "Enterovirus PCR result: ",
            "index": 74,
            "type": "string",
            "pattern": "^(Positive|Negative|Unknown)$"
        },
        "viral_result_6": {
            "description": "HSV skin and mucus membrane culture result: ",
            "index": 75,
            "type": "string",
            "pattern": "^(Positive|Negative|Unknown)$"
        },
        "viral_result_other": {
            "description": "Other viral culture or PCR result: ",
            "index": 76,
            "type": "string",
            "pattern": "^(Positive|Negative|Unknown)$"
        },
        "disch_date": {
            "description": "Date of hospital discharge: ",
            "index": 77,
            "type": "string"
        },
        "died_yn": {
            "description": "Did the infant die prior to NICU discharge? ",
            "index": 78,
            "type": "string",
            "pattern": "^(No|Yes|Unknown)$"
        },
        "disch_dispo": {
            "description": "Discharge disposition: ",
            "index": 79,
            "type": "string",
            "pattern": "^(Home|Unknown|Transfer\\ to\\ outside\\ NICU|Transfer\\ to\\ outside\\ step\\-down|Transfer\\ within\\ hospital\\ to\\ PICU|Transfer\\ within\\ hospital\\ to\\ step\\-down)$"
        },
        "death_cause": {
            "description": "Cause of death: ",
            "index": 80,
            "type": "string",
            "value-url": "http://purl.bioontology.org/ontology/LNC/LA10595-9"
        },
        "death_cause_other": {
            "description": "Cause of death if 'other': ",
            "index": 81,
            "type": "string"
        }
    },
    "type": "object",
    "additionalProperties": True,
    "required": [
        "subject_number",
        "demograph_site_num"
    ],
    "seperator": ",",
    "header": True,
    "examples": []
}

In [9]:
import json
import requests

root_url = "http://localhost:8080/"
response = requests.get(f"{root_url}datadownload/ark:99999/test-download/download")
response.content

b'{"detail":"Not Found"}'

In [3]:
import json
import requests

root_url = "http://localhost:8080/"
create_user1 = requests.post(root_url + "schema", data=json.dumps(schema))
create_user1.json()

{'created': {'@id': 'ark:59852/schema-predictive-modeling-izNjXSs',
  '@type': 'evi:Schema',
  'name': 'PreMo RedCAP Schema'}}

In [12]:
eg = {
    "@id":"ark:99999/sample-eg",
    "@type":"EVI:EvidenceGraph",
    "name":"Sample Evidence Graph",
    "description":"Fake graph for testing",
    "owner":"ark:99999/Me",
    "@graph":[
    {
    "name": "Output Data",
    "@type": "EVI:Dataset",
    "description": "An example output from a Compuation.",
    "@id": "Output Data",
    "generatedBy": {
        "name": "Fake Compuatation",
        "@type": "EVI:Computation",
        "@id": "Processing Compuation",
        "description": "Processing computation.",
        "usedDataset": [
            {
                "name": "Input Data",
                "@type": "EVI:Dataset",
                "description": "Fake Input Dataset.",
                "@id": "ark:99999/Input-Data"
            }
        ],
        "usedSoftware": [
            {
                "name": "Sample Software",
                "@type": "EVI:Software",
                "description": "Python processing script.",
                "@id": "ark:99999/fake-script"
            }
        ]
    }
}]}

In [19]:
create_eg = requests.post(root_url + "evidencegraph", data=json.dumps(eg))
create_eg.json()

{'error': 'evidencegraph already exists'}

In [26]:
rocrate_ark = "ark:59852/rocrate-premo-rocrate-eVHFS456"
og = requests.get(root_url + 'ark:59852/rocrate-premo-rocrate-eVHFS456').json()

In [27]:
og['hasEvidenceGraph'] = "ark:99999/sample-eg"

In [28]:
resp = requests.put(root_url + '/rocrate/ark:59852/rocrate-premo-rocrate-eVHFS456',data=json.dumps(og))
resp.json()

{'detail': 'Method Not Allowed'}